In [243]:
# try everything here
import sys
import networkx as nx # type: ignore
import graphviz
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from dowhy import CausalModel
import statsmodels.formula.api as smf

sys.path.append('../')
from src.data.make_dataset import load_data, merge_all_datasets

In [244]:
# Loading Data
data_dict = load_data()

olist_customers_df = data_dict['olist_customers_df']
olist_geolocation_df = data_dict['olist_geolocation_df']
olist_order_items_df = data_dict['olist_order_items_df']
olist_order_payments_df = data_dict['olist_order_payments_df']
olist_order_reviews_df = data_dict['olist_order_reviews_df']
olist_orders_df = data_dict['olist_orders_df']
olist_products_df = data_dict['olist_products_df']
olist_sellers_df = data_dict['olist_sellers_df']
product_category_name_translation_df = data_dict['product_category_name_translation_df']

olist_closed_deals_df = data_dict['olist_closed_deals_df']
olist_marketing_qualified_leads_df = data_dict['olist_marketing_qualified_leads_df']

In [245]:
df = merge_all_datasets(olist_customers_df, 
                       olist_geolocation_df,
                       olist_order_items_df,
                       olist_order_payments_df,
                       olist_order_reviews_df, 
                       olist_orders_df,
                       olist_products_df, 
                       olist_sellers_df
                       )

rainfall variable

In [246]:
rainfall_categories = {
    'North': {1: 'High', 2: 'High', 3: 'High', 4: 'Moderate', 5: 'Moderate', 6: 'Low', 7: 'Low', 8: 'Low', 9: 'Moderate', 10: 'Moderate', 11: 'High', 12: 'High'},
    'Northeast': {1: 'Low', 2: 'Moderate', 3: 'Moderate', 4: 'High', 5: 'High', 6: 'Moderate', 7: 'Low', 8: 'Low', 9: 'Low', 10: 'Moderate', 11: 'Moderate', 12: 'High'},
    'Central-West': {1: 'High', 2: 'High', 3: 'High', 4: 'Moderate', 5: 'Low', 6: 'Low', 7: 'Low', 8: 'Low', 9: 'Low', 10: 'Moderate', 11: 'High', 12: 'High'},
    'Southeast': {1: 'High', 2: 'High', 3: 'Moderate', 4: 'Moderate', 5: 'Low', 6: 'Low', 7: 'Low', 8: 'Low', 9: 'Moderate', 10: 'Moderate', 11: 'High', 12: 'High'},
    'South': {1: 'Moderate', 2: 'Moderate', 3: 'Moderate', 4: 'Moderate', 5: 'Moderate', 6: 'Low', 7: 'Low', 8: 'Low', 9: 'Moderate', 10: 'Moderate', 11: 'Moderate', 12: 'High'}
}

state_to_region = {
    'AC': 'North', 'AP': 'North', 'AM': 'North', 'PA': 'North', 'RO': 'North', 'RR': 'North', 'TO': 'North',
    'AL': 'Northeast', 'BA': 'Northeast', 'CE': 'Northeast', 'MA': 'Northeast', 'PB': 'Northeast', 'PE': 'Northeast', 'PI': 'Northeast', 'RN': 'Northeast', 'SE': 'Northeast',
    'GO': 'Central-West', 'MT': 'Central-West', 'MS': 'Central-West', 'DF': 'Central-West',
    'ES': 'Southeast', 'MG': 'Southeast', 'RJ': 'Southeast', 'SP': 'Southeast',
    'PR': 'South', 'RS': 'South', 'SC': 'South'
}



def get_rainfall_category(region, month):

    rainfall_categories = {
    'North': {1: 'High', 2: 'High', 3: 'High', 4: 'Moderate', 5: 'Moderate', 6: 'Low', 7: 'Low', 8: 'Low', 9: 'Moderate', 10: 'Moderate', 11: 'High', 12: 'High'},
    'Northeast': {1: 'Low', 2: 'Moderate', 3: 'Moderate', 4: 'High', 5: 'High', 6: 'Moderate', 7: 'Low', 8: 'Low', 9: 'Low', 10: 'Moderate', 11: 'Moderate', 12: 'High'},
    'Central-West': {1: 'High', 2: 'High', 3: 'High', 4: 'Moderate', 5: 'Low', 6: 'Low', 7: 'Low', 8: 'Low', 9: 'Low', 10: 'Moderate', 11: 'High', 12: 'High'},
    'Southeast': {1: 'High', 2: 'High', 3: 'Moderate', 4: 'Moderate', 5: 'Low', 6: 'Low', 7: 'Low', 8: 'Low', 9: 'Moderate', 10: 'Moderate', 11: 'High', 12: 'High'},
    'South': {1: 'Moderate', 2: 'Moderate', 3: 'Moderate', 4: 'Moderate', 5: 'Moderate', 6: 'Low', 7: 'Low', 8: 'Low', 9: 'Moderate', 10: 'Moderate', 11: 'Moderate', 12: 'High'}
    }

    return rainfall_categories.get(region).get(month, 'Unknown')


In [247]:
# Convert 'order_approved_at' column to datetime if it's not already
df['order_approved_at'] = pd.to_datetime(df['order_approved_at'])

# Extract the month from 'order_approved_at'
df['month'] = df['order_approved_at'].dt.month

# Map state to region
df['region'] = df['customer_state'].map(state_to_region)       # --------------> or seller_state?

# Map region to rainfall
df['rainfall_level'] = df.apply(lambda row: get_rainfall_category(row['region'], row['month']), axis=1)

weight variable

In [248]:
df['Product_weight_kg'] = df['product_weight_g']/1000

distance variable

In [249]:
# Define the Haversine function
def haversine(lat1, lon1, lat2, lon2):
    """
    Calculate the great-circle distance between two points
    on the Earth's surface given their latitude and longitude in decimal degrees.
    """
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    r = 6371  # Radius of Earth in kilometers
    distance = r * c

    return distance

# Example DataFrame
# df = pd.read_csv('path_to_your_data.csv')  # Replace with your actual data loading code

# Calculate distance for each row in the DataFrame
df['distance_km'] = df.apply(
    lambda row: haversine(
        row['geolocation_lat_x'], row['geolocation_lng_x'],
        row['geolocation_lat_y'], row['geolocation_lng_y']
    ),
    axis=1
)

product category variable

In [250]:
df['Product_category'] = df['product_category_name']

size variable

In [251]:
df['Product_size'] = df['product_length_cm'] * df['product_height_cm'] * df['product_width_cm']

### Previous rating

Rolling average of a customers previous rating (if they dont have previous orders it will be the same rating)

In [252]:
df = df.sort_values(by=['customer_id', 'review_answer_timestamp'])

# Create a new column for the rolling mean excluding the last review
def rolling_mean_excluding_last(group):
    return group['review_score'].shift(1).rolling(window=len(group) - 1).mean()

df['customer_avg_score'] = df.groupby('customer_id').apply(rolling_mean_excluding_last).reset_index(drop=True)

# If the customer has no previous scores, just use the current score
df['customer_avg_score'] = df['customer_avg_score'].fillna(df['review_score'])

/var/folders/7s/18428v6x0c9g1ltv9g71gm1h0000gn/T/ipykernel_44357/3204792575.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df['customer_avg_score'] = df.groupby('customer_id').apply(rolling_mean_excluding_last).reset_index(drop=True)


Rolling average of a sellers previous rating (if they dont have previous orders it will be the same rating)

In [253]:
# Previous Personal Purchase Experience

# Sort the dataframe by customer and order purchase timestamp
df = df.sort_values(by=['seller_id', 'order_purchase_timestamp'])

# Define a function to compute the rolling mean of previous review scores for each customer
def rolling_mean_previous_customer(group):
    # Shift to exclude the current review and calculate rolling mean of all previous reviews
    return group['review_score'].shift(1).expanding().mean()

# Apply the function to each group of customers
df['seller_hist_rating'] = df.groupby('seller_id').apply(rolling_mean_previous_customer).reset_index(drop=True)

# Handle cases where there are no previous reviews (NaN values) by setting them to 0 or any default value
df['seller_hist_rating'] = df['seller_hist_rating'].fillna(0)

/var/folders/7s/18428v6x0c9g1ltv9g71gm1h0000gn/T/ipykernel_44357/1062401438.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df['seller_hist_rating'] = df.groupby('seller_id').apply(rolling_mean_previous_customer).reset_index(drop=True)


No_photos variable

In [254]:
df['No_photos'] = df['product_photos_qty']

Price variable

In [255]:
df['Product_price'] = df['price']

Delivery late variable

In [256]:
df['order_delivered_customer_date'] = pd.to_datetime(df['order_delivered_customer_date'])
df['order_estimated_delivery_date'] = pd.to_datetime(df['order_estimated_delivery_date'])
df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'])

df['late_delivery_in_days'] = (df['order_delivered_customer_date'] - df['order_estimated_delivery_date']).dt.days

df['is_delivery_late'] = np.where(df['late_delivery_in_days'] > 0, 1, 0)

Rating Variable

In [257]:
df['Rating']= df['review_score']

Seasonality

In [258]:
df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'])
df['season'] = df['order_purchase_timestamp'].dt.month

In [ ]:
# hello

In [259]:
df_final = df[['order_id', 
               'payment_value', 
               'Rating',
               'rainfall_level',
               'region', 
               'Product_weight_kg', 
               'distance_km',
               'Product_category', 
               'Product_size', 
               'customer_avg_score',
               'No_photos',
               'Product_price', 
               'seller_hist_rating', 
               'season', 
               'is_delivery_late',
               'Product_price',
               'freight_value']]

In [140]:
df_final

,order_id,payment_value,Rating,rainfall_level,region,Product_weight_kg,distance_km,Product_category,Product_size,customer_avg_score,No_photos,Product_price,seller_hist_rating,season,is_delivery_late,Product_price
0,d455a8cb295653b55abda06d434ab492,916.02,5.0,Moderate,South,11.800,687.434185,eletroportateis,61920.0,5.0,2.0,895.0,0.000000,9,0,895.0
2,9dc8d1a6f16f1b89874c29c9d8d30447,916.02,5.0,Moderate,Southeast,11.800,714.295670,eletroportateis,61920.0,5.0,2.0,895.0,5.000000,10,0,895.0
1,7f39ba4c9052be115350065d07583cac,916.02,1.0,Moderate,Southeast,11.800,417.139650,eletroportateis,61920.0,1.0,2.0,895.0,5.000000,10,0,895.0
98,3c655487f0c8e34cde2c7b67de8f08cc,121.05,2.0,High,Central-West,8.875,1735.329124,ferramentas_jardim,24080.0,2.0,4.0,99.9,3.819149,2,0,99.9
26,eb188a175542057d90b3ca5628b7b5a0,573.85,4.0,High,Southeast,8.875,523.615247,ferramentas_jardim,24080.0,4.0,4.0,99.9,3.565217,2,0,99.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118427,392ed9afd714e3c74767d0c4d3e3f477,137.03,1.0,Unknown,Southeast,NaN,NaN,NaN,NaN,1.0,NaN,NaN,0.000000,9,0,NaN
118894,616fa7d4871b87832197b2a137a115d2,80.38,2.0,Unknown,Southeast,NaN,NaN,NaN,NaN,2.0,NaN,NaN,0.000000,10,0,NaN
118529,a2ac6dad85cf8af5b0afb510a240fe8c,197.55,1.0,Unknown,Southeast,NaN,NaN,NaN,NaN,1.0,NaN,NaN,0.000000,10,0,NaN
118704,b059ee4de278302d550a3035c4cdb740,222.03,5.0,Unknown,Northeast,NaN,NaN,NaN,NaN,5.0,NaN,NaN,0.000000,10,0,NaN


In [260]:
df_final.isnull().sum()

order_id                 0
payment_value            3
Rating                 997
rainfall_level           0
region                   0
Product_weight_kg      853
distance_km           1414
Product_category      2542
Product_size           853
customer_avg_score     896
No_photos             2542
Product_price          833
seller_hist_rating       0
season                   0
is_delivery_late         0
Product_price          833
freight_value          833
dtype: int64

In [261]:
df_final

,order_id,payment_value,Rating,rainfall_level,region,Product_weight_kg,distance_km,Product_category,Product_size,customer_avg_score,No_photos,Product_price,seller_hist_rating,season,is_delivery_late,Product_price,freight_value
0,d455a8cb295653b55abda06d434ab492,916.02,5.0,Moderate,South,11.800,687.434185,eletroportateis,61920.0,5.0,2.0,895.0,0.000000,9,0,895.0,21.02
2,9dc8d1a6f16f1b89874c29c9d8d30447,916.02,5.0,Moderate,Southeast,11.800,714.295670,eletroportateis,61920.0,5.0,2.0,895.0,5.000000,10,0,895.0,21.02
1,7f39ba4c9052be115350065d07583cac,916.02,1.0,Moderate,Southeast,11.800,417.139650,eletroportateis,61920.0,1.0,2.0,895.0,5.000000,10,0,895.0,21.02
98,3c655487f0c8e34cde2c7b67de8f08cc,121.05,2.0,High,Central-West,8.875,1735.329124,ferramentas_jardim,24080.0,2.0,4.0,99.9,3.819149,2,0,99.9,21.15
26,eb188a175542057d90b3ca5628b7b5a0,573.85,4.0,High,Southeast,8.875,523.615247,ferramentas_jardim,24080.0,4.0,4.0,99.9,3.565217,2,0,99.9,14.87
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118427,392ed9afd714e3c74767d0c4d3e3f477,137.03,1.0,Unknown,Southeast,NaN,NaN,NaN,NaN,1.0,NaN,NaN,0.000000,9,0,NaN,NaN
118894,616fa7d4871b87832197b2a137a115d2,80.38,2.0,Unknown,Southeast,NaN,NaN,NaN,NaN,2.0,NaN,NaN,0.000000,10,0,NaN,NaN
118529,a2ac6dad85cf8af5b0afb510a240fe8c,197.55,1.0,Unknown,Southeast,NaN,NaN,NaN,NaN,1.0,NaN,NaN,0.000000,10,0,NaN,NaN
118704,b059ee4de278302d550a3035c4cdb740,222.03,5.0,Unknown,Northeast,NaN,NaN,NaN,NaN,5.0,NaN,NaN,0.000000,10,0,NaN,NaN


In [241]:
df.columns

Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date',
       'order_item_id', 'product_id', 'seller_id', 'shipping_limit_date',
       'price', 'freight_value', 'payment_sequential', 'payment_type',
       'payment_installments', 'payment_value', 'review_id', 'review_score',
       'review_comment_title', 'review_comment_message',
       'review_creation_date', 'review_answer_timestamp',
       'product_category_name', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm',
       'customer_unique_id', 'customer_zip_code_prefix', 'customer_city',
       'customer_state', 'seller_zip_code_prefix', 'seller_city',
       'seller_state', 'geolocation_zip_code_prefix_x', 'geolocation_lat_x',
       'geolocation_lng_x', 'geo

In [89]:

model = smf.ols(formula='Rating ~ is_delivery_late', data=df_final)
results = model.fit()
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:                 Rating   R-squared:                       0.105
Model:                            OLS   Adj. R-squared:                  0.105
Method:                 Least Squares   F-statistic:                 1.391e+04
Date:                Sun, 22 Sep 2024   Prob (F-statistic):               0.00
Time:                        21:45:30   Log-Likelihood:            -2.0086e+05
No. Observations:              118146   AIC:                         4.017e+05
Df Residuals:                  118144   BIC:                         4.017e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            4.1328      0.004  

In [94]:
from statsmodels.miscmodels.ordinal_model import OrderedModel

In [262]:
df_final['Rating'] = df_final['Rating'].fillna(df_final['Rating'].mean())

/var/folders/7s/18428v6x0c9g1ltv9g71gm1h0000gn/T/ipykernel_44357/898008588.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['Rating'] = df_final['Rating'].fillna(df_final['Rating'].mean())


In [105]:

# Fit Ordinal Logistic Regression model
model = OrderedModel(df_final['Rating'], df_final['is_delivery_late'], distr='logit')
results = model.fit(method='bfgs')

# Print the summary of the results
print(results.summary())


Optimization terminated successfully.
         Current function value: 1.228526
         Iterations: 28
         Function evaluations: 31
         Gradient evaluations: 31
                             OrderedModel Results                             
Dep. Variable:                 Rating   Log-Likelihood:            -1.4637e+05
Model:                   OrderedModel   AIC:                         2.928e+05
Method:            Maximum Likelihood   BIC:                         2.928e+05
Date:                Sun, 22 Sep 2024                                         
Time:                        21:54:27                                         
No. Observations:              119143                                         
Df Residuals:                  119137                                         
Df Model:                           1                                         
                            coef    std err          z      P>|z|      [0.025      0.975]
---------------------------

In [106]:
# Fit Ordinal Logistic Regression model
model = OrderedModel(df_final['Rating'], df_final[['is_delivery_late', 'season']], distr='logit')
results = model.fit(method='bfgs')

# Print the summary of the results
print(results.summary())

Optimization terminated successfully.
         Current function value: 1.228392
         Iterations: 32
         Function evaluations: 36
         Gradient evaluations: 36
                             OrderedModel Results                             
Dep. Variable:                 Rating   Log-Likelihood:            -1.4635e+05
Model:                   OrderedModel   AIC:                         2.927e+05
Method:            Maximum Likelihood   BIC:                         2.928e+05
Date:                Sun, 22 Sep 2024                                         
Time:                        21:54:33                                         
No. Observations:              119143                                         
Df Residuals:                  119136                                         
Df Model:                           2                                         
                            coef    std err          z      P>|z|      [0.025      0.975]
---------------------------

In [263]:
from sklearn.preprocessing import LabelEncoder

le  = LabelEncoder()


df_final['Product_category_encoded'] = le.fit_transform(df_final['Product_category']) 


/var/folders/7s/18428v6x0c9g1ltv9g71gm1h0000gn/T/ipykernel_44357/2538155937.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['Product_category_encoded'] = le.fit_transform(df_final['Product_category'])


In [113]:
# Fit Ordinal Logistic Regression model
model = OrderedModel(df_final['Rating'], df_final[['is_delivery_late', 'Product_category_encoded', 'season']], distr='logit')
results = model.fit(method='bfgs')

# Print the summary of the results
print(results.summary())

Optimization terminated successfully.
         Current function value: 1.227953
         Iterations: 38
         Function evaluations: 41
         Gradient evaluations: 41
                             OrderedModel Results                             
Dep. Variable:                 Rating   Log-Likelihood:            -1.4630e+05
Model:                   OrderedModel   AIC:                         2.926e+05
Method:            Maximum Likelihood   BIC:                         2.927e+05
Date:                Sun, 22 Sep 2024                                         
Time:                        22:01:13                                         
No. Observations:              119143                                         
Df Residuals:                  119135                                         
Df Model:                           3                                         
                               coef    std err          z      P>|z|      [0.025      0.975]
------------------------

In [265]:
df_final.to_csv("../data/processed/data.csv", index=False)

In [242]:
df

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,...,Product_category,Product_size,customer_avg_score,seller_hist_rating,No_photos,Product_price,late_delivery_in_days,is_delivery_late,Rating,season
0,d455a8cb295653b55abda06d434ab492,944b72539d7e1f7f7fc6e46639ef1fe3,delivered,2017-09-26 22:17:05,2017-09-27 22:24:16,2017-09-29 15:53:03,2017-10-07 16:12:47,2017-10-30,1.0,a2ff5a97bf95719e38ea2e3b4105bce8,...,eletroportateis,61920.0,5.0,0.000000,2.0,895.0,-23.0,0,5.0,9
2,9dc8d1a6f16f1b89874c29c9d8d30447,d9442164acf4b03109425633efaa0cfc,delivered,2017-10-12 13:33:22,2017-10-12 13:49:22,2017-10-17 15:42:42,2017-10-24 20:17:44,2017-11-06,1.0,a2ff5a97bf95719e38ea2e3b4105bce8,...,eletroportateis,61920.0,5.0,5.000000,2.0,895.0,-13.0,0,5.0,10
1,7f39ba4c9052be115350065d07583cac,d7fc82cbeafea77bd0a8fbbf6296e387,delivered,2017-10-18 08:16:34,2017-10-18 23:56:20,2017-10-20 14:29:01,2017-10-27 16:46:05,2017-11-09,1.0,a2ff5a97bf95719e38ea2e3b4105bce8,...,eletroportateis,61920.0,1.0,5.000000,2.0,895.0,-13.0,0,1.0,10
98,3c655487f0c8e34cde2c7b67de8f08cc,5091c4ca04945991068cfba00258cc58,delivered,2017-02-04 19:06:04,2017-02-04 19:15:39,2017-02-10 09:24:35,2017-03-06 08:28:57,2017-03-13,1.0,e251ebd2858be1aa7d9b2087a6992580,...,ferramentas_jardim,24080.0,2.0,3.819149,4.0,99.9,-7.0,0,2.0,2
26,eb188a175542057d90b3ca5628b7b5a0,1eda56b47988d592fec945da9ee88c56,delivered,2017-02-18 23:26:24,2017-02-19 01:02:43,2017-02-21 07:07:39,2017-02-23 09:23:57,2017-03-21,1.0,e251ebd2858be1aa7d9b2087a6992580,...,ferramentas_jardim,24080.0,4.0,3.565217,4.0,99.9,-26.0,0,4.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118427,392ed9afd714e3c74767d0c4d3e3f477,2823ffda607a2316375088e0d00005ec,canceled,2018-09-29 09:13:03,NaT,NaN,NaT,2018-10-15,NaN,NaN,...,NaN,NaN,1.0,0.000000,NaN,NaN,NaN,0,1.0,9
118894,616fa7d4871b87832197b2a137a115d2,bf6181a85bbb4115736c0a8db1a53be3,canceled,2018-10-01 15:30:09,NaT,NaN,NaT,2018-10-23,NaN,NaN,...,NaN,NaN,2.0,0.000000,NaN,NaN,NaN,0,2.0,10
118529,a2ac6dad85cf8af5b0afb510a240fe8c,4c2ec60c29d10c34bd49cb88aa85cfc4,canceled,2018-10-03 18:55:29,NaT,NaN,NaT,2018-10-16,NaN,NaN,...,NaN,NaN,1.0,0.000000,NaN,NaN,NaN,0,1.0,10
118704,b059ee4de278302d550a3035c4cdb740,856336203359aa6a61bf3826f7d84c49,canceled,2018-10-16 20:16:02,NaT,NaN,NaT,2018-11-12,NaN,NaN,...,NaN,NaN,5.0,0.000000,NaN,NaN,NaN,0,5.0,10


In [ ]:
# customer_previous_score: better name would be customer_avg_score?
# There should be no capital letters on column names; convention
# let's make sure variable names are easy to understand; not sure what "rainfall" would mean if I had not contenxt